In [156]:
import pandas as pd
import re

In [157]:
石英砂 = False

In [158]:
附件1 = pd.read_excel('赛题/B/附件1.xlsx', '性能数据表')

附件1['催化剂组合编号'] = 附件1['催化剂组合编号'].ffill()
附件1['催化剂组合'] = 附件1['催化剂组合'].ffill()

old_columns = 附件1.columns
columns = ['乙醇转化率', '乙烯选择性', 'C4烯烃选择性', '乙醛选择性', '碳数为4-12脂肪醇选择性', '甲基苯甲醛和甲基苯甲醇选择性', '其他生成物的选择性']

附件1.rename(columns={old_columns[i+3]: columns[i] for i in range(len(columns))}, inplace=True)

rc = ['装料方式', 'Co', 'SiO2', 'HAP', '乙醇浓度']
if 石英砂:
    rc += '石英砂'

for i, c in enumerate(rc):
    附件1.insert(i + 3, c, 0)

def wt(desc, mode=False):
    coc = float(re.search(r'(?<= )\d+(\.\d+)?(?=wt)', desc).group()) / 100
    cosio2 = float(re.search(r'\d+(\.\d+)?(?=mg )', desc).group()) / 10**6
    if mode:
        co = cosio2 * coc
        sio2 = cosio2 * (1 - coc)
    else:
        co = cosio2 * coc / (coc + 1)
        sio2 = cosio2 * 1 / (coc + 1)
    return co, sio2

for i in 附件1.iterrows():
    附件1.loc[i[0], '温度'] = i[1]['温度'] + 273.15
    附件1.loc[i[0], '装料方式'] = 0 if i[1]['催化剂组合编号'][0] == 'A' else 1
    附件1.loc[i[0], 'Co'] ,附件1.loc[i[0], 'SiO2'] = wt(i[1]['催化剂组合'])
    if re.search(r'无HAP', i[1]['催化剂组合']):
        w = float(re.search(r'(?<= )\d+(\.\d+)?(?=mg石英砂)', i[1]['催化剂组合']).group()) / 10**6
        if 石英砂:
            附件1.loc[i[0], '石英砂'] = w
        else:
            附件1.loc[i[0], 'SiO2'] += w
    else:
        附件1.loc[i[0], 'HAP'] = float(re.search(r'(?<= )\d+(\.\d+)?(?=mg HAP)', i[1]['催化剂组合']).group()) / 10**6
    附件1.loc[i[0], '乙醇浓度'] = float(re.search(r'(?<=乙醇浓度)\d+(\.\d+)?(?=ml)', i[1]['催化剂组合']).group()) / 6 / 10**7
    for r in columns:
        附件1.loc[i[0], r] = i[1][r] / 100

附件1.drop(columns=['催化剂组合', '催化剂组合编号'], inplace=True)
附件1.to_pickle('附件1.pickle')
附件1.to_excel('附件1.xlsx')